Case Study: Sentiment Analysis of online reviews

While reviewing some NLP notes I've taken during an online course, I came across an intersting dataset which contains product reviews on Amazon. This particular subset of it conains reviews for samsung phones. I used to identify and extract features from text and transform them into feature vectors for the machine learning models. 

I used a logistic regression model to predict negative/positive reviews after using a Bag of Words model to generate features from text. 


Tags: Scikit-learn, Numpy and 

In [44]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/Amazon_Unlocked_Mobile.csv')
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [42]:
df.shape

(308277, 7)

### Pre-processing 

After droping missing values, I created a column that serves as the target value for the model. This is a binary value that is either 1 (Positively Rated) or 0 (Negatively Rated). I'm assuming here that any score below 3 out of 5 is negative. 

In [7]:
# drop rows with missing values
df.dropna(inplace=True)
# Assume ratings below 3 are un-neutral and remove them
df = df[df['Rating']!=3]
# create a new column that serves as a target for the model
df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)
df.head(10)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,1
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,1
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,1
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,1
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,1
5,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,1,I already had a phone with problems... I know ...,1.0,0
6,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,The charging port was loose. I got that solder...,0.0,0
7,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,"Phone looks good but wouldn't stay charged, ha...",0.0,0
8,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I originally was using the Samsung S2 Galaxy f...,0.0,1
11,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,This is a great product it came after two days...,0.0,1


Looking at the mean of the target value column, it is evident that the two classes in the set are inbalanced. The imabalce is not drastic as seen in Bank-Fraud detection datasets, where the class imbalance is much more significant because most of the entries charactrized as safe. 

This requires careful evaluation of the model, as using the accuracy metric alone will not reflect the true performance of the model. 

In [8]:
# mean of this column shows that we have inbalanced classes
df['Positively Rated'].mean()

0.7482686025879323

In [9]:
from sklearn.model_selection import train_test_split
# spliting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['Reviews'], df['Positively Rated'], random_state=0)

In [46]:
# checking a random query in the training set
print('X_train first entry:\n\n', X_train[75])
# what is the shape of the training set after the split
print('\n\nX_train shape:', X_train.shape)

X_train first entry:

 The keys are a little hard to hit, and I didn't expect a Spanish keyboard. But overall, a very good phone.


X_train shape: (231207,)


### Features Extraction

1. Bag of Words Model

Before the data is fed into an algorithm, it needs to be converted into a form that the algorithm can work with. i.e., the data needs to be represented numericlly. When dealing with text tasks, the **Bag of Words (BoW)** model is a popular approach. 

Using Scikit in Python we have access to `CountVectorizer` allows us to use BoW by convertng a collection of text documents into a matrix of words counts - works by first tokenizing the data and then building the vocabulary.

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

# creating an instance of the vectorizer, and fit it with training data
vect = CountVectorizer().fit(X_train)

After fitting the vectorizer with the training data, we can use it get useful information. 

In [20]:
# looking at every 2000 features -- small sense of what the vocav looks like -- messy
vect.get_feature_names()[::2000]

['00',
 '4less',
 'adr6275',
 'assignment',
 'blazingly',
 'cassettes',
 'condishion',
 'debi',
 'dollarsshipping',
 'esteem',
 'flashy',
 'gorila',
 'human',
 'irullu',
 'like',
 'microsaudered',
 'nightmarish',
 'p770',
 'poori',
 'quirky',
 'responseive',
 'send',
 'sos',
 'synch',
 'trace',
 'utiles',
 'withstanding']

To get a snse of what the vocabulary looks like, I'm looking at stepping randomly into the data. I see words like `00`, `adr6275`, `4less`, it seems a little messy. 

In [21]:
# how many features?
len(vect.get_feature_names())

53216

Using the vectorizer's `transform()` method on the training data, I converted it to a `document-term matrix`, whcih I find the [Wikipedia definion](https://en.wikipedia.org/wiki/Document-term_matrix) of, very consise. 

In [22]:
# Producing the document-term matrix 
X_train_vectorized = vect.transform(X_train)
X_train_vectorized

<231207x53216 sparse matrix of type '<class 'numpy.int64'>'
	with 6117776 stored elements in Compressed Sparse Row format>

The result is a very large **sparse-matrix** where each row corrosponds to a document and each column is a word from our training vocabolary. The entries in the matrix represents the number of time each word appears in each documents, which explains why the matrix is sparse (i.e., Most entries are zero). The number of words is way larger than the number of words that might appear in an Amazon single review. 

This matrix can now be used to learn a model using **logistic regression**, which is a good option for high-dimensional datasets that are sparse. It is also very easy to train and interpret. 

In [23]:
from sklearn.linear_model import LogisticRegression

# Train the model
model = LogisticRegression().fit(X_train_vectorized, y_train)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Since the dataset contains classes that are imbalanced, it is not a good idea to use evaluation metric like the simple accuracy metric. 

In [24]:
from sklearn.metrics import roc_auc_score

# Predict the transformed test documents
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.9206361352734463


The AUC score of the logistic model is 92% when tested against the test set. Note that before obtaining the predictions, the vectorizer object needs to be transformed with the test set as well. 

In [25]:
# get the feature names as numpy array
feature_names = np.array(vect.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['worst' 'garbage' 'junk' 'unusable' 'false' 'worthless' 'useless'
 'crashing' 'disappointing' 'awful']

Largest Coefs: 
['excelent' 'excelente' 'exelente' 'loving' 'loves' 'perfecto' 'excellent'
 'complaints' 'awesome' 'buen']


Looking at the 10 smallest and 10 largets model cofficients after sorting them, it is evident that the model has connected words like disappointing, garbage, and unusable to negative reviews. And words like excellent, love and amazing ot positive reviews. The 10 largest coefficients are being indexed using [:-11:-1] so the list returned is in order of largest to smallest

2. Tf-idf 

Term Frequency - Inverse Document Frequency [6]. Allows to weight words in terms of how important they are in a document where high weight is given to terms that appear frequently in a particular document, but don't appear much in the corpus. 

Features with low tfidf are either commonly used across all documents or rarely used and only occur in long documents. features with high tf-idf are frequently used within specific documents but rarely across all documents

because it uses the same tokenization technique used by CountVectorizer, it will return the same number of features**



**a good way of reducing the number of features, and might help reduce overfitting when using both CountVectorizer and tdidf, by passing `min_df` argument, which specifies the minimum number a word has to appear in a document to be considered**. 


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names())

17951

Looking at the number of features now it down to 17951 from 53216 after adding said condition above. 

Applying LR to the tfidf again by transforming the training and testing data, fiting the model and obtaining predictions. It shows that the same performance can be obtained using a smaller number of features. 

In [31]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.9265848398605042


looking at the list of smallest and largest tfidf. the list of small ones, either commonly appeared in all reviews, or rarely appeared in very long reviews. what is the relationship with the length of the review largesT: appeared frequently in a review, but not commonly across all reiews


In [32]:
# Storing feature names in a Numpy array
feature_names = np.array(vect.get_feature_names())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest tfidf:
['commenter' 'pthalo' 'warmness' 'storageso' 'aggregration' '1300'
 '625nits' 'a10' 'submarket' 'brawns']

Largest tfidf: 
['defective' 'batteries' 'gooood' 'epic' 'luis' 'goood' 'basico'
 'aceptable' 'problems' 'excellant']


In [33]:
sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['not' 'worst' 'useless' 'disappointed' 'terrible' 'return' 'waste' 'poor'
 'horrible' 'doesn']

Largest Coefs: 
['love' 'great' 'excellent' 'perfect' 'amazing' 'awesome' 'perfectly'
 'easy' 'best' 'loves']


In the previous BoW model, the actual words order within sentences was not taken into consideration. That is to say, there is no context, meaning. A demonstration of this would be to compare two sentences, with the same vocabulary, but contain different meaning. 

In [47]:
# These reviews are treated the same by our current model
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]


**Adding n-grams**
One way of fixing this is is by adding a sense of context to the model. This is done by adding **sequences of word features** known as n-grams. For example a bi-grams gives pairs of adjacent words. This is conveniently done by passing an n-gram argument to the `CountVectorizer`.


In [35]:
# min document frequency of 5 and extracting 1-grams and 2-grams
vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

X_train_vectorized = vect.transform(X_train)
# now checking the number of words 
len(vect.get_feature_names())

198917

The number of features has increased to ~ 200 thousands. Although n-grams are powerful at capturing meaning, long sequences can cause an explosion to the number of features.

Now I'm going to check the performance of an logistic regression model with using the vectorizer with the added n-grams. As seen below, the accuracy did improve to 95% from 92%. 

In [36]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.9594770780721797


In [38]:
# check out what bi-grams are associated with negative and positive reviews
feature_names = np.array(vect.get_feature_names())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['no good' 'not happy' 'not worth' 'junk' 'worst' 'not satisfied'
 'garbage' 'not good' 'defective' 'terrible']

Largest Coefs: 
['excelent' 'excelente' 'excellent' 'not bad' 'exelente' 'perfect'
 'awesome' 'no problems' 'no issues' 'perfecto']


Performing the same test I did earlier to check the logic of the model regarding negative and positive reviews. It is now returning the desired behavior. 

In [39]:
# Reviews are now correctly identified
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]


Vectorizers in Python are very flexibile and support many other tasks  like removing stop words and lemmatization [1]. 

# Refereneces


[1. Scikit-learn](https://scikit-learn.org/)

[2](https://docs.scipy.org/doc/numpy/reference/)

[3] (https://www.coursera.org/learn/python-text-mining)

[4] Applied text Mining in Python course https://www.coursera.org/learn/python-text-mining

[5] https://en.wikipedia.org/wiki/Document-term_matrix
[6] http://www.tfidf.com/